- Knock-in payoff is given as below
$$F(S) = exp(-rT)\cdot 1000\cdot I(S_T &gt; K)\cdot I(min_{1 \leq k \leq m} S (t_k) &lt; H) $$

In [5]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt

- GBM class

In [6]:
class GBM():
    def __init__(self, Drift, Vol, InitState):
        self.Drift = Drift #scalar
        self.Vol = Vol #scalar
        self.InitState = InitState
        self.Mu = lambda x, t: Drift * x
        self.Sigma = lambda x, t: Vol * x
        
    def _Wn(self,T,m):
        t=np.linspace(0,T,m+1)
        Wh=np.zeros(m+1)
        for i in range(m):
            DeltaW = np.sqrt(t[i+1] - t[i]) * np.random.normal()
            Wh[i+1] = Wh[i] + DeltaW
        return Wh
        
    def _explicit_sol_(self, t, W_t):
        x0 = self.InitState
        b = self.Drift
        sigma = self.Vol
        return x0 * np.exp((b - sigma**2/2.) * t + sigma * W_t)

- Main function

In [7]:
if __name__ == "__main__":
    r=0.05
    vol=0.15
    S0=95
    iGBM = GBM(r,vol,S0)
    T=0.25
    m=50
    H=85
    K=96
    k=1000
    n=1000

    Price=np.zeros(n)
    for j in range(n):
        payoff=0
        for i in range(k):
            W_t=iGBM._Wn(T,m)
            t=np.linspace(0,T,m+1)
            St=iGBM._explicit_sol_(t,W_t)
 
            if St[-1]>K and min(St)<H:
                payoff=payoff+1000
                #print(payoff)
        #price=payoff/k
        Price[j]=payoff/k*np.exp(-r*T)
        #print(j)
    print('The final price is ',np.mean(Price))
    plt.plot(np.linspace(0,n,n),Price)
    plt.show()

    mean=np.mean(Price)
    print('The mean is ',mean)
    std=np.sqrt(np.mean((Price-np.mean(Price)**2)))
    print('The standard is ',std)
    print('The 95 percent confidence interval is between %4.3f and %4.3f' %(mean-2*std,mean+2*std))

Knock-in payoff is given as below
$$F(S) =exp(-rT) \cdot 1000\cdot I(S_T &gt; K)\cdot I(min_{1 \leq k \leq m} S (t_k) &lt; H) $$
Crude Monte Carlo simulation can be sampled from the below normal random distribution f(x)
$$ S_t = S_0 * exp(L_n)  $$$$ L_n = \sum_{i=1}^{m}X_i  $$$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t,\sigma^2 \delta t)  $$
If Importance Sampling (IS) is adopted, the alternative normal random distribution g(x) and the likelihood ratio are as follows
$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t-b,\sigma^2 \delta t)  $$$$ \frac {f(x_1,x_2,...,x_m)}{g(x_1,x_2,...,x_m)} = exp(-\frac{1}{2}\sum_{k=1}^{m}{\frac{(x_k-\mu)}{\sigma \sqrt{\delta t}}}^2) \cdot exp(\frac{1}{2}\sum_{k=1}^{m}{\frac{(x_k-\mu+b)}{\sigma \sqrt{\delta t}}}^2)
= exp( \frac{b}{\sigma ^2 \delta t}\sum_{k=1}^{m}x_k-\frac{mb}{\sigma^2}(r-\frac{\sigma^2}{2})+\frac{mb^2}{2 \sigma^2 \delta t}     )                                                                            $$
The payoff is as below (The sampling should follow the alternative pdf g(x))
$$ exp(-rT)\cdot \frac {f(x_1,x_2,...,x_m)}{g(x_1,x_2,...,x_m)} \cdot F^{g(x)}(S)$$

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats

- Input paramters

In [0]:
#risk-free rate
r=0.05
#volatility
vol=0.15
#initial price
S0=95
#maturity
T=0.25
#time steps
m=50
#barrier price
H=85
#maturity price
K=96
#number of MC simulation
k=1000
#simulation time for each MC
n=1000
#time interval
dt=T/m

- equivalent mu and sig
- b value determined from the following equation $$ b=\frac{1}{m} log(\frac{S_0}{H}) $$

In [0]:
b=math.log(S0/H)/m
c=math.log(K/S0)
mudt2=(r-0.5*vol**2)*dt-b
mudt1=(r-0.5*vol**2)*dt

In [0]:
sidt=vol*np.sqrt(dt)

NCrossed=np.zeros(k)

#Payoff=np.zeros(k)
Times=np.zeros(k)
StockVals=np.zeros(k)
ISRatio=np.zeros(k)

Ncrossed=0
TBreach=0
Price=np.zeros(n)

for counter in range(n):
    Payoff=np.zeros(k)
    for i in range(k):
        vetZ=np.random.normal(mudt2,sidt,m)
        ##vetZ2=np.hstack()
        vetZ2=np.hstack((math.log(S0),vetZ))
        LogPath=np.cumsum(vetZ2)
        St=np.exp(LogPath)


        jcross=0
        for j in range(len(St)):
            if St[j]<H:
                jcross=j
                break
            #print(jcross)
            #the path crosses the lower barrier
        if jcross>0:
                
            #TBreach=jcross-1
            #Times[Ncrossed]=TBreach*dt
            #StockVals[Ncrossed]=St[jcross]
            #ISRatio[Ncrossed]=np.exp(TBreach*b**2/2/vol**2/dt+b/vol**2/dt*np.sum(vetZ[1:TBreach])-TBreach*b/vol**2*(r-vol**2/2))
            ISRatio[Ncrossed]=np.exp(m*b**2/2/vol**2/dt+b/vol**2/dt*np.sum(vetZ)-m*b/vol**2*(r-vol**2/2))
                
            if St[-1]>K:
                Payoff[Ncrossed]=ISRatio[Ncrossed]*1000*np.exp(-r*T)
                Ncrossed=Ncrossed+1
        
    Price[counter]=np.sum(Payoff)/k   
    #print(counter)    
print('The final payoff is: ',np.mean(Price))

In [0]:
plt.plot(np.linspace(0,n,n),Price)
plt.show()

In [4]:
mean=np.mean(Price)
print('The mean is ',mean)
std=np.sqrt(np.mean((Price-np.mean(Price)**2)))
print('The standard is ',std)
print('The 95 percent confidence interval is between %4.3f and %4.3f' %(mean-2*std,mean+2*std))

The mean is  0.49082616684545916
The standard is  0.4999158337012867
The 95 percent confidence interval is between -0.509 and 1.491
